In [4]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.decomposition import PCA
from sklearn import svm
import seaborn as sns
from matplotlib.colors import ListedColormap
from sklearn import neighbors
import matplotlib.pyplot as plt
from pandas.plotting import parallel_coordinates
from sklearn.preprocessing import StandardScaler
from sklearn import decomposition
import scipy
from sklearn.manifold import TSNE
from pandas.plotting import parallel_coordinates
from sklearn import decomposition
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.decomposition import PCA
from sklearn import svm
import seaborn as sns
from matplotlib.colors import ListedColormap
from sklearn import neighbors
import math
from scipy import stats
import matplotlib.pyplot as plt
from pandas.plotting import parallel_coordinates
from sklearn.preprocessing import StandardScaler
from sklearn import decomposition
import scipy
from math import  pi, exp,log, sqrt
from datetime import date
import time
from scipy.stats import norm
from sklearn.manifold import TSNE
import seaborn as sn
from sklearn.datasets import make_friedman1
from sklearn.decomposition import SparsePCA
from dask import dataframe as dd
import signal

In [5]:
OP_df = pd.read_csv('Option2017_2_clean.csv')
OP_df = OP_df.dropna()
OP_df.head(3)

,Unnamed: 0,currentDate,ExpDate,StrikePrice,Ticker,Type,Last,Implied volatility,Underlaying asset price,Expiration time
0,0,2/17/17,7/21/17,34.799999,TBNK,Call,1.06,0.13250,33.549999,0.421629
1,2,2/17/17,3/17/17,22.500000,ARR,Call,0.08,0.13956,21.709999,0.076660
2,3,2/17/17,3/17/17,20.000000,ARR,Put,0.20,0.32783,21.709999,0.076660


In [10]:
class callV():
    
    def __init__(self,last, S, K, r, T, sigma, tol = 0.000000000001):
        
        self.last = last
        
        self.S = S
        self.K = K
        
        self.r = r
        self.T = T
        
        self.sigma = sigma
        self.tol = float(tol)
        
    def d11(self,sigma):
    
        x = ( log (self.S/self.K) + (( self.r + ((sigma**2)/2) )* self.T ))/( sigma * ( sqrt ( self.T )))
        return x 

    def d22(self,sigma):
    
        x = ( log (self.S/self.K) + (( self.r - ((sigma**2)/2) )* self.T ))/( sigma * ( sqrt ( self.T )))
        return x
    
    def call(self,sigma):

        return self.S  * norm.cdf( self.d11(sigma) ) - self.K * (exp(- (self.r*self.T) ) ) * norm.cdf( self.d22(sigma))

    def newcall(self, sigma = None):
        
        if sigma == None:
            sigma = self.sigma

        d1 = self.d11(sigma)
        d2 = self.d22(sigma)

        return self.call(sigma)
    
    def func(self, sigma = None):
        
        return self.newcall(sigma) - self.last
    
    def Vega(self):

        d111 = self.d11(self.sigma)
    
        vega = self.S  * np.sqrt(self.T) * norm.pdf(d111)
        return vega

    def Vomma(self):
        
        d1 = self.d11(self.sigma)
        d2 = self.d22(self.sigma)
        return (self.Vega()*d1*d2) / self.sigma


    def hallVol(self):
        i = 0
        #print(i)
        while self.func() > self.tol:
            i = i + 1
            self.sigma = self.sigma + ( sqrt( (self.Vega()**2)  - (2*self.func()*self.Vomma() ) ) -self.Vega() )/self.Vomma()
            if(i>1000):
                break
        return self.sigma,i


Implied Volatility 0.11907144846731929
Iteration 3


In [12]:
class putV():
    
    def __init__(self,last, S, K, r, T, sigma, tol = 0.000000000001):
        
        self.last = last
        
        self.S = S
        self.K = K
        
        self.r = r
        self.T = T
        
        self.sigma = sigma
        self.tol = float(tol)
        
    def d11(self,sigma):
    
        x = ( log (self.S/self.K) + (( self.r + ((sigma**2)/2) )* self.T ))/( sigma * ( sqrt ( self.T )))
        return x 

    def d22(self,sigma):
    
        x = ( log (self.S/self.K) + (( self.r - ((sigma**2)/2) )* self.T ))/( sigma * ( sqrt ( self.T )))
        return x
    
    def put(self,sigma):

        return  -self.S  * norm.cdf(- self.d11(sigma) ) + self.K * (exp(- (self.r*self.T) ) ) * norm.cdf(- self.d22(sigma))

    def newcall(self, sigma = None):
        
        if sigma == None:
            sigma = self.sigma

        d1 = self.d11(sigma)
        d2 = self.d22(sigma)

        return self.put(sigma)
    
    def func(self, sigma = None):
        
        return self.newcall(sigma) - self.last
    
    def Vega(self):

        d111 = self.d11(self.sigma)
    
        vega = self.S  * np.sqrt(self.T) * norm.pdf(d111)
        return vega

    def Vomma(self):
        
        d1 = self.d11(self.sigma)
        d2 = self.d22(self.sigma)
        return (self.Vega()*d1*d2) / self.sigma


    def hallVol(self):
        i = 0
        #print(i)
        while self.func() > self.tol:
            i = i + 1
            self.sigma = self.sigma + ( sqrt( (self.Vega()**2)  - (2*self.func()*self.Vomma() ) ) -self.Vega() )/self.Vomma()
            if(i>1000):
                break
        return self.sigma,i


In [6]:
S = np.array( OP_df["Underlaying asset price"]).astype("float")
K = np.array( OP_df["StrikePrice"]).astype("float")
r = .03
T = np.array( OP_df["Expiration time"]).astype("float")
Op = np.array( OP_df["Last"]).astype("float")
Type = np.array( OP_df["Type"]).astype("str")
OP_df[1446:].head(3)

,Unnamed: 0,currentDate,ExpDate,StrikePrice,Ticker,Type,Last,Implied volatility,Underlaying asset price,Expiration time
1446,1588,2/17/17,3/17/17,825.0,GOOG,Call,13.32,0.10210,828.070007,0.07666
1447,1589,2/17/17,3/17/17,830.0,GOOG,Call,10.60,0.11668,828.070007,0.07666
1448,1590,2/17/17,3/17/17,835.0,GOOG,Call,8.60,0.13335,828.070007,0.07666


In [9]:
i=1448
m,it = callV( Op[i] , S[i] , K[i] , r , T[i],.31 ).hallVol()
print("Implied Volatility",m)
print("Iteration",it)

Implied Volatility 0.11907144846731929
Iteration 3


In [10]:
imp = []
iter = []
for i in range( 0, len(OP_df) ):
    if(Type[i] == "Call"):
        try:
            #print(i)
            m,it = (callV( Op[i] , S[i] , K[i] , r , T[i],.31 ).hallVol()) 
            imp.append(m)
            iter.append(it)
        except:
            imp.append(.31)
            iter.append(100)
    else:
        try:
            #print(i)
            m,it = (putV( Op[i] , S[i] , K[i] , r , T[i],.31 ).hallVol()) 
            imp.append(m)
            iter.append(it)
        except:
            imp.append(.31)
            iter.append(100)

In [11]:
OP_df['imp'] = pd.Series(imp)
OP_df['iter'] = pd.Series(iter)
OP_df.head(3)

,Unnamed: 0,currentDate,ExpDate,StrikePrice,Ticker,Type,Last,Implied volatility,Underlaying asset price,Expiration time,imp,iter
0,0,2/17/17,7/21/17,34.799999,TBNK,Call,1.06,0.13250,33.549999,0.421629,0.162619,3
1,2,2/17/17,3/17/17,22.500000,ARR,Call,0.08,0.13956,21.709999,0.076660,0.132739,3
2,3,2/17/17,3/17/17,20.000000,ARR,Put,0.20,0.32783,21.709999,0.076660,0.310000,0


In [12]:
OP_df["diff"] = OP_df['Implied volatility'] - OP_df['imp']
OP_df['mse'] = OP_df['diff']**2
mse = OP_df['mse'].sum()/len(OP_df)
iteration = OP_df['iter'].sum()/len(OP_df)

In [13]:
n = 1 / ( ( 1 + mse ) * log( 1 + iteration ) )
n
n = 1 / ( ( 1 + mse ) * log( 1 + iteration ) )
print("MSE :",mse)
print("Average Iteration : ",iteration)
print("Average Efficiency",n)

MSE : 0.03949312084290312
Average Iteration :  3.351705069124424
Average Efficiency 0.6541741011527735


In [14]:
list = []
list.append(mse)
list.append(iteration)
list.append(n)


In [15]:
df = pd.read_csv('result.csv')

In [16]:
df["Halley-Irrational"] = list
df.head(3)

,bisection,Brent,Newton-Raphson,New-Newton,Muller-Bisection,New-Halley,Halley-Irrational
0,0.031976,0.032722,0.029877,0.029913,0.030416,0.025485,0.039493
1,41.053819,16.120337,3.642949,9.617389,26.130753,6.207127,3.351705
2,0.259168,0.340924,0.632422,0.410988,0.294026,0.493729,0.654174


In [17]:
df.to_csv('result.csv', index=False)